# Prüfung - Praktisches Web Scraping

## Vorstellung

Sie sind ein Student des digitalen Designs und interessieren sich für Geschäftsstrategien. Daher bewahren Sie immer einen Ordner mit den aktuellsten Artikeln auf, den Sie offline lesen können, beispielsweise wenn Sie während Ihres Sommerurlaubs vom Stromnetz getrennt sind.

Dazu müssen Sie die Kategorieseite „Strategie“ einer Ihrer Lieblingswebsites, **Fast Company**, durchsuchen.

Nutzen Sie das Wissen, das Sie bisher über Pythons Listen-, Wörterbuch- und String-Methoden gesammelt haben, `for`-Schleifen, `if`-Anweisung, kombiniert mit `.select()` / `.select_one()`, `.find()` / `.find_all()`, `.parent`, `.text` und andere nützliche BeautifulSoup-Methoden zur Lösung der folgenden Herausforderung. 

Viel Spaß!

## Einstellung

Verwenden Sie die URL unten, um die Nachrichtenartikel zu scrapen.

In [20]:
url_fc_category = "https://web.archive.org/web/20250822090500/https://www.fastcompany.com/section/strategy"

## Durchführung

Schließen Sie das Miniprojekt ab und übermitteln Sie Ihren Code und Ihre Ergebnisse.

### Teil 1

1. Scrapen Sie die Artikelkategorieseite mit der angegebenen URL.
2. Suchen Sie den Abschnitt „Latest Strategy News“ und extrahieren Sie Titel, URL, Miniaturansicht, Beschreibungstext und Veröffentlichungsdatum des Artikels in ein Wörterbuch.
3. Speichern Sie das Wörterbuch in einer Liste.

Beispiel für das Format der Liste:

```python
list = [
    {
        "url": "...",
        "title": "...",
        "description": "...",
        "thumbnail": {
            "url": "...",
            "alt": "..."
        },
        "category": {
          "url": "...",
          "text": "..."
        }
    },
    ...
]
```

In [ ]:
# Schreiben Sie hier Ihre Lösung für Teil 1






































### Teil 2

Teil 2 setzt den vorherigen Teil fort.

1. Holen Sie sich den ersten Nachrichtenartikel aus der Liste der neuesten Artikel, die Sie in Teil 1 gespeichert haben.
2. Extrahieren Sie den Titel des Artikels und seinen Inhalt.
2. Formatieren Sie die Daten und speichern Sie sie als gültige Markdown-Datei.

**Wichtig**: Denken Sie daran, immer darauf zu achten, dass die URL, mit der Sie arbeiten, korrekt formatiert ist. Wenn sich die URL hinter einer Archivierungslösung wie der **Wayback Machine** befindet, möchten Sie möglicherweise zuerst die URL bereinigen und den folgenden Teil herausnehmen: `https://web.archive.org/web/20250822090500/`.

In [ ]:
# Schreiben Sie hier Ihre Lösung für Teil 2


































## Bewertungskriterien

Ihre Leistung wird nach folgenden Kriterien bewertet:

- **Aufgabenerledigung:** Alle Aufgaben werden wie beschrieben erledigt.
- **Korrektheit:** Code läuft fehlerfrei und erzeugt die erwartete Ausgabe.

### Bestehen des Kurses

Der geschriebene Code muss nicht schön aussehen, sollte aber die erwarteten Ergebnisse und Ausgaben liefern. Der Lernende kann folgende Konzepte und Ansätze erfolgreich anwenden:

- Holen Sie sich den Quellcode der Website; Headers bei Bedarf senden.
- Betrachten Sie die Website visuell und den entsprechenden Quellcode, um die besten Orientierungspunkte und Elemente zu finden, die zum Scrapen der Informationen verwendet werden könnten
- Verwenden Sie die entsprechenden Funktionen und Eigenschaften, die auf BeautifulSoup verfügbar sind, um die Daten zu extrahieren
- Gehen Sie den DOM-Baum auf und ab
- Verwenden Sie `for`-Schleife, `if`-Anweisungen, Listen, Worterbücher, JSON um korrektes Code zu erzeugen.


## Lösungen

### Teil 1

```python
import requests
from bs4 import BeautifulSoup

headers = {
  'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36',
  'Accept-Language': 'en-US,en;q=0.9',
  'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8'
}

category_page = requests.get(url_fc_category)
soup = BeautifulSoup(category_page.text, 'html.parser')

latest_news_header = soup.find('h2')
latest_articles = latest_news_header.parent.find('section').find_all('article')

latest_articles_list = []

for article in latest_articles:
  article_category = article.contents[0]
  article_info = article.contents[1]

  article_details = {
    'url': article_info.get('href', ''),
    'title': article_info.find_all('p')[0].text,
    'description': article_info.find_all('p')[1].text,
    'thumbnail': {
      "url": article_info.find('img').get('src', ''),
      'alt': article_info.find('img').get('alt', '')
    },
    'category': {
      'url': article_category.find('a').get('href', ''),
      'text': article_info.find_all('p')[0].text
    }
  }
  
  latest_articles_list.append(article_details)
```

### Teil 2

```python
latest_article_url = latest_articles_list[0]['url']
clean_url = 'https://' + latest_article_url.split('https://').pop()

headers = {
  'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36',
  'Accept-Language': 'en-US,en;q=0.9',
  'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8'
}

article_page = requests.get(clean_url, headers=headers)
soup2 = BeautifulSoup(article_page.text, 'html.parser')

article_title = soup2.h1.text.strip()
article_elements = soup2.select_one('article').select('h2, p')

article_content = []

for element in article_elements:
  if element.name == 'h2':
    article_content.append(f'## {element.text.strip()}')
  elif element.text.strip() == '|':
    continue
  else:
    article_content.append(''.join([str(subelement) for subelement in element.contents]))

article_content = '\n\n'.join(article_content)

filename = clean_url.split('/').pop()
with open(f'{filename}.md', 'w', encoding='utf-8') as file:
  file.write(f'# {article_title}\n')
  file.write('\n')
  file.write(article_content)
```